In [5]:
import os
import urllib.request
from tempfile import TemporaryDirectory

# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

#Date time
import datetime as dt
from datetime import timedelta, date

# Visualization
import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd
import geopandas
from rasterio import features
import rasterio


# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix

# Planetary Computer Tools
import pystac
import pystac_client
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc
import contextily
pc.settings.set_subscription_key('b0067a12405d4fd4a4cc82d28869d9bc')

# Others
import requests
import rich.table
from itertools import cycle
from tqdm import tqdm
tqdm.pandas()

#Scaling using Dask
import dask_gateway
import dask
import stackstac
import xrspatial.multispectral as ms
from dask_gateway import GatewayCluster

In [6]:
crop_presence_data = pd.read_csv("./Data/Crop_Location_Data_20221201.csv")
crop_presence_data.head()

,Latitude and Longitude,Class of Land
0,"(10.323727047081501, 105.2516346045924)",Rice
1,"(10.322364360592521, 105.27843410554115)",Rice
2,"(10.321455902933202, 105.25254306225168)",Rice
3,"(10.324181275911162, 105.25118037576274)",Rice
4,"(10.324635504740822, 105.27389181724476)",Rice


#### Defining resolution, scale and time of interest

In [7]:
time_of_interest = '2021-12-01/2022-12-01'
resolution = 10  # meters per pixel 
scale = resolution / 111320.0 # degrees per pixel for crs=4326

In [14]:
def create_dataset_from_sentinel2(bbox):
    stac = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=pc.sign_inplace,
    )

    search = stac.search(
        bbox=bbox,
        datetime= time_of_interest,
        collections=["sentinel-2-l2a"],
        query={"eo:cloud_cover": {"lt": 30}},
    )

    items = search.item_collection()
    print(len(items))
    data = (
        stackstac.stack(
            items,
            assets=["B04", "B03", "B02"],  # red, green, blue
            chunksize=10240,
            epsg=4326,
            resolution=scale
        )
        .where(lambda x: x > 0, other=np.nan)  # sentinel-2 uses 0 as nodata
        .assign_coords(band=lambda x: x.common_name.rename("band"))  # use common names
    )
    
    return data

In [9]:
def calculate_bbox(lat_long, box_size_deg=0.0004):
    lat_long=lat_long.replace('(','').replace(')','').replace(' ','').split(',')
    
    min_lon = float(lat_long[1]) - box_size_deg/2
    min_lat = float(lat_long[0])- box_size_deg/2
    max_lon = float(lat_long[1]) + box_size_deg/2
    max_lat = float(lat_long[0]) + box_size_deg/2
    
    return min_lon, min_lat, max_lon, max_lat

In [10]:
rvi_df = pd.DataFrame()
bbox = []
for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
    bbox.append(calculate_bbox(coordinates))

bbox_data = pd.DataFrame(bbox,columns =['min_lon', 'min_lat', 'max_lon', 'max_lat'])

100%|████████████████████████████████████████████████████████████████████████████████████████| 600/600 [00:00<?, ?it/s]


'''time_slice = "2021-12-01/2022-12-01"
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
             (10.726539, 105.142254), 
             (10.603391, 105.062709),
             (10.337941, 104.977229),
             (10.188656, 105.168294),
             (10.335607, 105.503777),
        ]
    ],
}

bbox = rasterio.features.bounds(area_of_interest)'''

In [15]:
for i in range(2):
    data= create_dataset_from_sentinel2(bbox_data.iloc[i])
    print(data)

12
<xarray.DataArray 'stackstac-b39516e9d21e4db58b5b3ac745fd48fc' (time: 12,
                                                                band: 3,
                                                                y: 11073,
                                                                x: 11184)>
dask.array<where, shape=(12, 3, 11073, 11184), dtype=float64, chunksize=(1, 1, 10240, 10240), chunktype=numpy.ndarray>
Coordinates: (12/46)
  * time                                     (time) datetime64[ns] 2021-12-06...
    id                                       (time) <U54 'S2A_MSIL2A_20211206...
  * band                                     (band) <U5 'red' 'green' 'blue'
  * x                                        (x) float64 105.0 105.0 ... 106.0
  * y                                        (y) float64 10.86 10.86 ... 9.861
    s2:thin_cirrus_percentage                (time) float64 13.76 ... 11.33
    ...                                       ...
    title                             

In [19]:
data

<xarray.DataArray 'stackstac-e915a3010634784fafbdc96e019dc7d7' (time: 12,
                                                                band: 3,
                                                                y: 11073,
                                                                x: 11184)>
dask.array<where, shape=(12, 3, 11073, 11184), dtype=float64, chunksize=(1, 1, 10240, 10240), chunktype=numpy.ndarray>
Coordinates: (12/46)
  * time                                     (time) datetime64[ns] 2021-12-06...
    id                                       (time) <U54 'S2A_MSIL2A_20211206...
  * band                                     (band) <U5 'red' 'green' 'blue'
  * x                                        (x) float64 105.0 105.0 ... 106.0
  * y                                        (y) float64 10.86 10.86 ... 9.861
    s2:thin_cirrus_percentage                (time) float64 13.76 ... 11.33
    ...                                       ...
    title                                    (band) <U20 'Band 4 - Red - 10m'...
    proj:shape                               object {10980}
    common_name                              (band) <U5 'red' 'green' 'blue'
    center_wavelength                        (band) float64 0.665 0.56 0.49
    full_width_half_max                      (band) float64 0.038 0.045 0.098
    epsg                                     int32 4326
Attributes:
    spec:        RasterSpec(epsg=4326, bounds=(104.99973050664751, 9.86085159...
    crs:         epsg:4326
    transform:   | 0.00, 0.00, 105.00|\n| 0.00,-0.00, 10.86|\n| 0.00, 0.00, 1...
    resolution:  8.983111749910169e-05

In [47]:
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [-122.2751, 47.5469],
            [-121.9613, 47.9613],
            [-121.9613, 47.9613],
            [-122.2751, 47.9613],
            [-122.2751, 47.5469],
        ]
    ],
}

time_range = "2020-12-01/2020-12-31"

search = catalog.search(
    collections=["landsat-c2-l2"], intersects=area_of_interest, datetime=time_range
)

In [3]:
search

NameError: name 'search' is not defined